作業一 房價預測

import需要的套件，然後讀取資料，然後觀察一下資料，確定這是一筆漂亮的資料。

In [9]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split

df=pd.read_csv("housing.csv")


df.describe().T



,count,mean,std,min,25%,50%,75%,max
Avg. Area Income,5000.0,6.858311e+04,10657.991214,17796.631190,61480.562388,6.880429e+04,7.578334e+04,1.077017e+05
Avg. Area House Age,5000.0,5.977222e+00,0.991456,2.644304,5.322283,5.970429e+00,6.650808e+00,9.519088e+00
Avg. Area Number of Rooms,5000.0,6.987792e+00,1.005833,3.236194,6.299250,7.002902e+00,7.665871e+00,1.075959e+01
Avg. Area Number of Bedrooms,5000.0,3.981330e+00,1.234137,2.000000,3.140000,4.050000e+00,4.490000e+00,6.500000e+00
Area Population,5000.0,3.616352e+04,9925.650114,172.610686,29403.928702,3.619941e+04,4.286129e+04,6.962171e+04
Price,5000.0,1.232073e+06,353117.626581,15938.657923,997577.135049,1.232669e+06,1.471210e+06,2.469066e+06


這是最一開始的程式碼，先把X和y定義出來，變數 X 包含資料集中的前5個特徵，用於建立機器學習模型來預測目標變數；y代表從資料集中選取名為 'Price' 的那一列。

然後把資料拆成訓練的和測試的，X,y,test_size=0.3代表有30%的資料用在測試，也就是70%用在訓練。reg=LinearRegression()就是線性回歸模型，reg.fit(X_train, y_train)就是在找訓練數據的最佳線性關係。接下來就是預測，然後測模型的預測準測度。(這段是打給之後的我看的，怕我忘記順便解釋一下。)


In [10]:
X=df.iloc[:,:5]
y=df['Price']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=54)

from sklearn.linear_model import LinearRegression

reg=LinearRegression()

reg.fit(X_train,y_train)

predictions=reg.predict(X_test)

from sklearn.metrics import r2_score

r2_score(y_test,predictions)

0.921660486570713

接下來就是我如何提高這個r2_score，我在了解完整個程式到底在幹嘛後，第一個想法就是更改測試和訓練數據的比例，反正要嘛調高要嘛調低，一定會調到一個r2score數字比較高的一邊。

下面先把用於測試的比例調高。

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.31,random_state=54)

reg=LinearRegression()

reg.fit(X_train,y_train)

predictions=reg.predict(X_test)

r2_score(y_test,predictions)

0.9209470198448942

數字變小了，那調低吧。

In [34]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.29,random_state=54)

reg=LinearRegression()

reg.fit(X_train,y_train)

predictions=reg.predict(X_test)

r2_score(y_test,predictions)

0.9227374820848911

好跟預想的一樣，0.9227374820848911>0.921660486570713了，那無聊一下，試著把數據切割到提升r2score小數點後四位好了。

In [57]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.282,random_state=54)

reg=LinearRegression()

reg.fit(X_train,y_train)

predictions=reg.predict(X_test)

r2_score(y_test,predictions)

0.923030226599351

事實證明是在進行到小數點後第四位的數據比例調整後，r2_score的分數就不會有甚麼變化了，(我測試了0.281~0.289，到了0.282用於測試時r2 score到達最高，後面要測試0.2821和0.2819，發現0.2821變低，0.2819的r2score跟0.282一樣，我猜它是自己四捨五入了吧。)

然後就是要尋找一下其他方法來提升r2score了，我找到了正則化這個方法。

In [65]:
from sklearn.linear_model import Lasso, Ridge

# 使用Lasso正則化
lasso_reg = Lasso(alpha=0.1)  # 設定alpha參數，alpha越大，正則化強度越高
lasso_reg.fit(X_train, y_train)
lasso_predictions = lasso_reg.predict(X_test)

r2_score(y_test,lasso_predictions)


0.9230302347673092

In [66]:
# 使用Ridge正則化
ridge_reg = Ridge(alpha=0.1)  # 設定alpha參數，alpha越大，正則化強度越高
ridge_reg.fit(X_train, y_train)
ridge_predictions = ridge_reg.predict(X_test)

r2_score(y_test,ridge_predictions)

0.9230303221358307

r2score是都有比一開始高啦，但我覺得其實也沒有明顯提升，在看完資料後，才真正了解甚麼是正則化和甚麼是過擬合，我覺得沒什麼提升是因為這個房價的資料太少特徵(Lasso)，然後可能也沒什麼過擬合的問題(Ridge)，大概吧。

然後我查到了隨機森林回歸，想試試看。

In [67]:
from sklearn.ensemble import RandomForestRegressor

# 創建隨機森林回歸模型
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)  # 這裡的參數可以根據需要調整

# 使用訓練數據擬合模型
rf_regressor.fit(X_train, y_train)

# 進行預測
rf_predictions = rf_regressor.predict(X_test)

r2_score(y_test,rf_predictions)

0.8910052630545354

再讓森林長大一點好了。

In [69]:
from sklearn.ensemble import RandomForestRegressor

# 創建隨機森林回歸模型
rf_regressor = RandomForestRegressor(n_estimators=10000, random_state=42)  # 這裡的參數可以根據需要調整

# 使用訓練數據擬合模型
rf_regressor.fit(X_train, y_train)

# 進行預測
rf_predictions = rf_regressor.predict(X_test)

r2_score(y_test,rf_predictions)

0.8924661772140354

森林太大了，然後好像沒什麼用。

最後貼上我的參考連結，但我最好的課後老師名字叫做Chatgpt，大部分的問題都是問它的，謝謝這個偉大的語言模型。

https://blog.csdn.net/qq_39276337/article/details/123783999
https://juejin.cn/s/sklearn%20r2_score
https://juejin.cn/post/6844904018309824519